# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234567989945                   -0.50    8.0
  2   -7.250283641363       -1.80       -1.41    1.0
  3   -7.251049665235       -3.12       -1.99    1.0
  4   -7.250969370305   +   -4.10       -1.88    2.0
  5   -7.251335740565       -3.44       -2.77    1.0
  6   -7.251338430435       -5.57       -3.40    1.0
  7   -7.251338723566       -6.53       -3.57    2.0
  8   -7.251338791097       -7.17       -4.05    1.0
  9   -7.251338797581       -8.19       -4.67    1.0
 10   -7.251338798545       -9.02       -4.85    3.0
 11   -7.251338798659       -9.95       -5.18    1.0
 12   -7.251338798688      -10.53       -5.39    2.0
 13   -7.251338798702      -10.85       -5.99    1.0
 14   -7.251338798704      -11.71       -6.30    3.0
 15   -7.251338798705      -12.51       -6.99    2.0
 16   -7.251338798705      -14.27       -7.55    1.0
 17   -7.251338798705      -15.05       -7.79 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.034184863026181254
[ Info: Arnoldi iteration step 2: normres = 0.5319725171416453
[ Info: Arnoldi iteration step 3: normres = 0.2471089116383626
[ Info: Arnoldi iteration step 4: normres = 0.9359558122835486
[ Info: Arnoldi iteration step 5: normres = 0.5808161881623259
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.80e-01, 8.47e-02, 2.55e-01, 1.81e-01, 4.15e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4648591697894539
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.06e-01, 8.56e-02, 3.86e-01, 1.03e-01, 4.19e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09346727050181008
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.05e-02, 1.46e-02, 3.14e-02, 4.53e-02, 6.35e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10816648518912077
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.82e-